In [ ]:
from datetime import datetime

# Funcion para la busqueda de estación más parecida a una dada
def findSimilarStationRanking(station, continent_table, bbdd_cursor):
    ranking=DataFrame()

    # En primer lugar se leen todas las estaciones del hemisferio sur (latitud negativa)
    df_hemis_sur = pd.read_csv("D:/master/Master-in-Data-Science/Proyecto/Datos/ghcnd-stations-cluster.csv",
                                sep=";",decimal=",")
    df_hemis_sur['station'] = df_hemis_sur.COUNTRY + df_hemis_sur.ID
    df_hemis_sur = df_hemis_sur[df_hemis_sur.LATITUDE < 0][['station']]
    
    # En primer lugar buscamos la estacion en el continente indicado
    rows = bbdd_cursor.execute("select date, TMAX, TMIN, PRCP from "+continent_table+" where station = '"+station+"';")
    
    is_south_master = False
    if max(df_hemis_sur.station.isin([station])):
        is_south_master = True
    
    result_north=[]
    result_south=[]
    for r in rows:
        # Se convierte la fecha a formato fecha
        if is_south_master:
            date_south = r[0]
            date_north = datetime.strptime(r[0], '%Y%m%d') - pd.DateOffset(months=6)
            #date_north = date_north.strftime("%Y%m%d")
            date_north = str(date_north.year) + str(date_north.month) + str(date_north.day)
        else:
            date_north = r[0]
            date_south = datetime.strptime(r[0], '%Y%m%d') + pd.DateOffset(months=6)
            #date_south = date_south.strftime("%Y%m%d")
            date_south = str(date_south.year) + str(date_south.month) + str(date_south.day)
                        
                        
        result_north.append({"date":date_north,"ROOT_TMAX":r[1],"ROOT_TMIN":r[2],"ROOT_PRCP":r[3]})        
        result_south.append({"date":date_south,"ROOT_TMAX":r[1],"ROOT_TMIN":r[2],"ROOT_PRCP":r[3]})        

    df_root_station_north = DataFrame(result_north)
    df_root_station_south = DataFrame(result_south)
    result_north = []
    result_south = []

    # Para cada continente, se itera por pais, para recuperar y comparar la informacion de cada estacion
    continentes=["antarctica","central_america","south_america","north_america","europa","africa","asia","oceania"]
    for continente in continentes:  
        print ("Iterando con el continente " + continente)
        rows = bbdd_cursor.execute("""select station, date, TMAX, TMIN, PRCP 
                    from world_time_"""+continente+" order by station;")
        
        n = 0
        result = []
        for r_stat in rows:
            # Se recuperan todas las observaciones de cada estacion
            s = r_stat[0]
            if n == 0:
                current_station = s
                
            date = r_stat[1]      
            tmax = r_stat[2]
            tmin = r_stat[3]
            prcp = r_stat[4]
                
            n = n + 1

            # Se compara con la estacion maestra, si es igual, no se hace nada, en otro caso se construye el
            # DataFrame con el detalle de las observaciones
            if current_station != station:                
                if s == current_station:
                    # Seguimos construyendo el DataFrame de la estacion
                    result.append({"station":s, "date":date,"TMAX":tmax,"TMIN":tmin,"PRCP":prcp})
                else:
                    # Llegamos a la primera observacion de la siguiente estacion
                    # El conjunto de observaciones de la estacion previa se convierten en un DataFrame
                    df_station = DataFrame(result)
                    df_station = df_station.fillna(0)
        
                    # Con la estacion recuperada, realizamos el merge con la estacion buscada
                    df_station_north = df_station.merge(df_root_station_north[["date","ROOT_TMAX","ROOT_TMIN","ROOT_PRCP"]],
                                                        on='date')
                    df_station_south = df_station.merge(df_root_station_south[["date","ROOT_TMAX","ROOT_TMIN","ROOT_PRCP"]],
                                                        on='date')
                    df_station=DataFrame()

                    # Se calcula la diferencia entre cada una de las medidas con los datos hemisferio norte
                    df_station_north["DIFF_TMAX"] = pow(df_station_north["ROOT_TMAX"]-df_station_north["TMAX"],2)
                    df_station_north["DIFF_TMIN"] = pow(df_station_north["ROOT_TMIN"]-df_station_north["TMIN"],2)
                    df_station_north["DIFF_PRCP"] = pow(df_station_north["ROOT_PRCP"]-df_station_north["PRCP"],2)
                    df_station_north["NRECORDS"] = 1
                    
                    df_station_north = df_station_north[["station","date","DIFF_TMAX","DIFF_TMIN","DIFF_PRCP","NRECORDS"]]
                    df_station_north = df_station_north.groupby("station").sum()
                    df_station_north.reset_index()
                    
                    df_station_north["TMAX"] = pow(df_station_north["DIFF_TMAX"]/df_station_north["NRECORDS"],0.5)
                    df_station_north["TMIN"] = pow(df_station_north["DIFF_TMIN"]/df_station_north["NRECORDS"],0.5)
                    df_station_north["PRCP"] = pow(df_station_north["DIFF_PRCP"]/df_station_north["NRECORDS"],0.5)
                    df_station_north['SIMILARIDAD'] = (df_station_north['TMAX']+df_station_north['TMIN'])/2
                    #df_station_north = df_station_north[["station","TMAX","TMIN","PRCP","NRECORDS"]]

                    
                    # Se calcula la diferencia entre cada una de las medidas con los datos hemisferio sur
                    df_station_south["DIFF_TMAX"] = pow(df_station_south["ROOT_TMAX"]-df_station_south["TMAX"],2)
                    df_station_south["DIFF_TMIN"] = pow(df_station_south["ROOT_TMIN"]-df_station_south["TMIN"],2)
                    df_station_south["DIFF_PRCP"] = pow(df_station_south["ROOT_PRCP"]-df_station_south["PRCP"],2)
                    df_station_south["NRECORDS"] = 1
    
                    df_station_south = df_station_south[["station","date","DIFF_TMAX","DIFF_TMIN","DIFF_PRCP","NRECORDS"]]
                    df_station_south = df_station_south.groupby("station").sum()
                    df_station_south.reset_index()
                
                    df_station_south["TMAX"] = pow(df_station_south["DIFF_TMAX"]/df_station_south["NRECORDS"],0.5)
                    df_station_south["TMIN"] = pow(df_station_south["DIFF_TMIN"]/df_station_south["NRECORDS"],0.5)
                    df_station_south["PRCP"] = pow(df_station_south["DIFF_PRCP"]/df_station_south["NRECORDS"],0.5)
                    df_station_south['SIMILARIDAD'] = (df_station_south['TMAX']+df_station_south['TMIN'])/2
                    #df_station_south = df_station_south[["station","TMAX","TMIN","PRCP","NRECORDS"]]
                    
                    # Se guardan los resultados de la comparativa

                    if len(df_station_north) > 0 and len(df_station_south > 0):
                        if df_station_north.SIMILARIDAD[0] <= df_station_south.SIMILARIDAD[0]:            
                            ranking = ranking.append(df_station_north)
                        else:
                            ranking = ranking.append(df_station_south)
                        
                    df_station_north = DataFrame()
                    df_station_south = DataFrame()
                   
                    # Se inicializan las variables para que funcione correctamente el bucle
                    result = []
                    current_station = s                    
                    result.append({"station":s, "date":date,"TMAX":tmax,"TMIN":tmin,"PRCP":prcp})
            
    ranking = ranking.sort_values(by="SIMILARIDAD")
    print (ranking.head())
    ranking.to_csv(station+"_similarity.csv")

In [ ]:
import sqlite3
import pandas as pd
from pandas import DataFrame,Series
# Abrimos la conexión con la BBDD
conn = sqlite3.connect("D:/master/Master-in-Data-Science/Proyecto/Datos/worldTimeSeries.sqlite")
cursor = conn.cursor()

findSimilarStationRanking('ITE00100554', 'world_time_europa', cursor)

conn.close()